In [ ]:
import sandy
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

In [ ]:
tape = sandy.get_endf6_file("endfb_71", 'nfpy', 'all')
fy = sandy.Fy.from_endf6(tape)

In [ ]:
energy = 'thermal'
zam = 922350
chy = sandy.fy.get_chain_yields().query(f"E=='{energy}' & ZAM=={zam}").set_index("A")
Vy_extra = sandy.CategoryCov.from_stdev(chy.DCHY).data
cov = fy.gls_cov_update(
    zam=zam,
    e=2.53e-2,
    Vy_extra=Vy_extra,
    kind="mass yield",
    rows=None,
    decay_data=None,
    threshold=None,
)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 6), dpi=100)

corr = cov.get_corr()

ax = axs[0]
ax.set_aspect('equal')
sns.heatmap(corr, vmin=-1, vmax=1, cmap="bwr", ax=ax)
ax.set_title('correlation matrix')

ax = axs[1]
ax.set_aspect('equal')
ax.spy(corr)
ax.set_title('SPY matrix')

fig.tight_layout()

In [ ]:
e = 2.53000e-02
fy_u235t = fy.data.query(f"E=={e} & ZAM=={zam} & MT==454")
fy_dict = fy_u235t.set_index("ZAP").FY.to_dict()

std_prior = fy_u235t[["ZAP", "DFY"]].reset_index(drop=True).assign(kind="prior")
std_post = pd.Series(np.sqrt(np.diag(cov.data)), index=cov.data.index).rename("DFY").reset_index().assign(kind="posterior")
std = pd.concat([std_prior, std_post], ignore_index=True)
std["FY"] = std.ZAP.apply(lambda x: fy_dict[x])

z_a_m = pd.DataFrame(
    map(sandy.shared.expand_zam, std.ZAP),
    columns=["Z", "A", "M"],
    dtype=int,
)
std = std.merge(z_a_m, left_index=True, right_index=True)

std = std.loc[std.FY!=0]  # remove zeros
std["DFY"] *= 100 / std["FY"]

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), dpi=100)
sns.scatterplot(data=std, x="Z", y="DFY", hue="kind", style="kind", size="A")
ax.set_ylabel("indepedendent FY uncertainty / %")
fig.tight_layout()